# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [2]:
users_be = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
users_be.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
users_be.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Заменим messages на тип int

In [5]:
users_be['messages'] = users_be['messages'].astype('int64')

Заменим calls на тип int

In [6]:
users_be['calls'] = users_be['calls'].astype('int64')

In [7]:
users_be.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null int64
minutes     3214 non-null float64
messages    3214 non-null int64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


## Разбейте данные на выборки

In [8]:
# <Поделим датафрейм на обучающую выборку и выборку, которую позже разделим на валидационную и тестовую.>
df_train, df_divide = train_test_split(users_be, test_size=0.40, random_state=12345)

# <Поделим df_divide валидационную и тестовую выборку.>
df_valid, df_test = train_test_split(df_divide, test_size=0.50, random_state=12345)

## Исследуйте модели

In [9]:
# <Разделим обучающий датафрейм на features и target - целевой признак>
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

# <Разделим валидационный датафрейм на features и target - целевой признак>
features_valid = df_valid.drop(['is_ultra'], axis=1)  
target_valid = df_valid['is_ultra']

Начнем с исследования модели случайного леса принятия решений.

Оценим в цикле долю правильных ответов для разных глубин дерева принятия решений:

In [10]:
best_RF = None
best_accuracy_RF = 0
best_est_RF = 0
best_depth_RF = 0
for est in range(2,51):
    for depth in range(2,25):
        RF = RandomForestClassifier(random_state = 40,n_estimators = est, max_depth = depth)
        RF.fit(features_train,target_train)
        prediction_valid_RF = RF.predict(features_valid)
        accuracy_RF = accuracy_score(target_valid, prediction_valid_RF)
        if best_accuracy_RF < accuracy_RF:
            best_RF = RF
            best_depth_RF = depth
            best_est_RF = est
            best_accuracy_RF = accuracy_RF

Теперь исследуем модели дерева принятия решений.

Оценим в цикле долю правильных ответов для разных глубин дерева принятия решений:

In [11]:
best_DT = None
best_accuracy_DT = 0
best_depth_DT = 0
for depth in range(2,50):
    DT = DecisionTreeClassifier(random_state = 40, max_depth = depth)
    DT.fit(features_train,target_train)
    prediction_valid_DT = DT.predict(features_valid)
    accuracy_DT = accuracy_score(target_valid, prediction_valid_DT)
    if best_accuracy_DT < accuracy_DT:
        best_DT = DT
        best_accuracy_DT = accuracy_DT
        best_depth_DT = depth

Теперь исследуем модели логистической регрессии.

Оценим в цикле долю правильных ответов для разных глубин дерева принятия решений:

In [12]:
LR = LogisticRegression(random_state = 40,max_iter= 1000,tol=1e-5,solver = 'lbfgs' )
LR.fit(features_train,target_train)
prediction_valid_LR = LR.predict(features_valid)
accuracy_LR = accuracy_score(target_valid, prediction_valid_LR)

Посмотрим на все результаты

In [13]:
print(f"Logistic Regression: {accuracy_LR}")
print(f"Random Forest: {best_accuracy_RF}")
print(f"Decision Tree: {best_accuracy_DT}")

Logistic Regression: 0.7107309486780715
Random Forest: 0.807153965785381
Decision Tree: 0.7869362363919129


Основываясь на показателе точности, можно прийти к выводу, что наиболее качественные результаты прогнозирует Random Forest

## Проверьте модель на тестовой выборке

In [14]:
# <Разделим тестовый датафрейм на features и target - целевой признак>
features_test = df_test.drop(['is_ultra'], axis=1)  
target_test = df_test['is_ultra']

In [15]:
prediction_test_LR = LR.predict(features_test)
prediction_test_RF = best_RF.predict(features_test)
prediction_test_DT = best_DT.predict(features_test)

In [16]:
accuracy_LR_test = accuracy_score(prediction_test_LR, target_test)
accuracy_RF_test = accuracy_score(prediction_test_RF, target_test)
accuracy_DT_test = accuracy_score(prediction_test_DT, target_test)

In [17]:
print(f"Logistic Regression: {accuracy_LR_test}")
print(f"Random Forest: {accuracy_RF_test}")
print(f"Decision Tree: {accuracy_DT_test}")

Logistic Regression: 0.6842923794712286
Random Forest: 0.8102643856920684
Decision Tree: 0.7838258164852255


Наиболее адекватные предсказания делает Random Forest, это видно по точности предсказания на валидационной и на тестовой выборке

## (бонус) Проверьте модели на адекватность

Для проверки адекватности нашей модели напишем модель (функцию), предсказывающую значение признака is_ultra самым простым способом, 50 на 50, и сравним, лучше ли наша модель:

In [18]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy='most_frequent', random_state=12345).fit(features_train, target_train)
dummy_pred = dummy.predict(features_test)
print('Accuracy Dummy-модели на тестовой выборке:', accuracy_score(target_test, dummy_pred))

accuracy = 0.4914


In [ ]:
(target_test==0).sum() / len(target_test)